In [2]:
!nvidia-smi

Thu Nov 16 09:46:46 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 6.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 80.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 3.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
from datasets import load_dataset, Dataset
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# 기존 데이터셋 불러오기
ds = load_dataset("beomi/KoAlpaca-v1.1a", split="train")

# 새로운 데이터 로딩 (예: pandas DataFrame)
new_data = pd.read_csv("/content/drive/MyDrive/alpaca_data.csv", encoding='cp949')

# pandas DataFrame을 datasets의 Dataset으로 변환
new_dataset = Dataset.from_pandas(new_data)

# 두 데이터셋을 pandas DataFrame으로 변환
ds_df = ds.to_pandas()
new_dataset_df = new_dataset.to_pandas()

# 두 데이터프레임을 합치기
combined_df = pd.concat([ds_df, new_dataset_df])

# 다시 Dataset으로 변환
combined_dataset = Dataset.from_pandas(combined_df)

# 결과 확인
print(combined_dataset)

data = combined_dataset.map(
    lambda x: {'text': f"### 질문: {x['instruction']}\n\n### 답변: {x['output']}<|endoftext|>" }
)

print(data)


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/21155 [00:00<?, ? examples/s]

Dataset({
    features: ['instruction', 'output', 'url', '__index_level_0__'],
    num_rows: 22005
})


Map:   0%|          | 0/22005 [00:00<?, ? examples/s]

Dataset({
    features: ['instruction', 'output', 'url', '__index_level_0__', 'text'],
    num_rows: 22005
})


In [4]:
model_id = "EleutherAI/polyglot-ko-12.8b"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

data = data.map(lambda samples: tokenizer(samples["text"]), batched=True)

(…)12.8b/resolve/main/tokenizer_config.json:   0%|          | 0.00/174 [00:00<?, ?B/s]

(…)lot-ko-12.8b/resolve/main/tokenizer.json:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

(…).8b/resolve/main/special_tokens_map.json:   0%|          | 0.00/204 [00:00<?, ?B/s]

(…)lyglot-ko-12.8b/resolve/main/config.json:   0%|          | 0.00/678 [00:00<?, ?B/s]

(…)esolve/main/model.safetensors.index.json:   0%|          | 0.00/52.5k [00:00<?, ?B/s]

...

model-00027-of-00028.safetensors:   0%|          | 0.00/896M [00:00<?, ?B/s]

model-00028-of-00028.safetensors:   0%|          | 0.00/518M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/28 [00:00<?, ?it/s]

(…)2.8b/resolve/main/generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Map:   0%|          | 0/22005 [00:00<?, ? examples/s]

In [5]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [6]:
from peft import LoraConfig, get_peft_model

def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )


config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 6553600 || all params: 6608701440 || trainable%: 0.09916622894073424


In [ ]:
import transformers

# needed for gpt-neo-x tokenizer
tokenizer.pad_token = tokenizer.eos_token

trainer = transformers.Trainer(
    model=model,
    train_dataset=data,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=1,
        # warmup_steps=200,
        max_steps=500, ## 초소형만 학습: 10 step = 20개 샘플만 학습.
        learning_rate=2e-4,
        fp16=True,
        logging_steps=10,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,2.430200
20,2.149400
30,2.041000
40,1.944300
50,1.895600
60,1.877300
70,1.977300
80,1.925400
90,1.920400
100,1.881100


In [8]:
model.eval()
model.config.use_cache = True  # silence the warnings. Please re-enable for inference!

model.generate(**tokenizer("### 질문: 오늘 날씨는?", return_tensors='pt', return_token_type_ids=False))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1353: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1671: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


tensor([[    6,     6,     6,  2438,    29,  1832,  4770,   272,    34,   224,
           202, 23849,   301,   768,   283,  1219,  3200,    15,  1832,  4770]])

In [24]:
def gen(x):
    gened = model.generate(
        **tokenizer(
            f"### 질문: {x}\n\n### 답변:",
            return_tensors='pt',
            return_token_type_ids=False
        ),
        max_new_tokens=256,
        early_stopping=True,
        do_sample=True,
        eos_token_id=2,
    )
    print(tokenizer.decode(gened[0]))

In [10]:
# 질문만한 경우
gen("""
Rubella 항체 결합력 검사의 급여 기준은 무엇인가요?
""")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:422: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 
Rubella 항체 결합력 검사의 급여 기준은 무엇인가요?


### 답변: Rubella 항체 결합력 검사의 급여 기준은 다음과 같습니다.
#### Rubella 항체 결합력 검사를 시행할 경우 주의해야 할 사항 1) 면역억제제 사용으로 면역형광값이 낮은 대상(예시: 항암치료 환자, 장기이식 후 이식거부를 예방하기 위한 이식 후 환자 등)인 경우, 2) 면역억제제를 사용하지 않는 환자이지만 면역형광값이 낮은 경우, 
3) 1일 용량 면역형광값이 20 IU/ml 이하인 경우, 
4) 임신한 경우, 
5) 면역억제제를 사용한 경우에도 면역형광값이 낮다면 검사는 금기입니다.

#### Rubella 항체 결합력 검사 비용(선택적 사용) 
1) 면역형광검사 + 면역화학검사 : 70,100원(선택적 사용, 면역형광검사로도 진단이 가능하지만 검사결과를 확인하고자 할 경우 면역형광검사와 면역화학검사를 병행해 사용하면 판단에 도움됨)
2) 염색체 검사 : 45,150원(염색체 이상을 감별


In [11]:
# 정보를 제공하고 질문한 경우
gen("""
다음 내용을 보고 질문에 답변해줘

누654나(1)주 항체 결합력검사의 급여기준
누654나(1)주. 항체 결합력 검사는 각 검사항목의 특이 IgM 항체가 양성 또는 특이 IgM 항체가 음성이지만 IgG 항체가 증가되어 급성기감염이 의심되는 경우에 인정함.
※ 보건복지부 고시 제2021-183호(2021.7.1. 시행)
■ 개정 사유: 문구변경

Rubella 항체 결합력 검사의 급여 기준은 무엇인가요?
""")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 
다음 내용을 보고 질문에 답변해줘

누654나(1)주 항체 결합력검사의 급여기준
누654나(1)주. 항체 결합력 검사는 각 검사항목의 특이 IgM 항체가 양성 또는 특이 IgM 항체가 음성이지만 IgG 항체가 증가되어 급성기감염이 의심되는 경우에 인정함.
※ 보건복지부 고시 제2021-183호(2021.7.1. 시행)
■ 개정 사유: 문구변경

Rubella 항체 결합력 검사의 급여 기준은 무엇인가요?


### 답변: 

1. 
누654나(1) 항체 결합력검사는 특이 IgM 항체가 양성인 경우에 인정되며, IgG 항체가 증가되어 급성기감염이 의심되는 경우에 한하여 급여가 인정됩니다.
2. 누654나(1)주. 항체 결합력검사의 급여기준

(1) 항체 결합력 검사는 각 검사항목의 특이 IgM 항체가 양성 또는 특이 IgM 항체가 음성이지만 IgG 항체가 증가되어 급성기감염이 의심되는 경우에 인정함. 
3. 따라서, 해당 항원에 항체가 형성된 사람은 양성반응자에 대해 추가 검사를 시행할 필요가 없으며, 급성감염의 경우가 아니라면 음성판정 후 추가 검사를 실시할 필요가 없습니다.�
추가 답변이 필요하면 추가질문 또는 1:1질문을 달아주세요!

한상원 내과 전문의였습니다. 최근 2형 당뇨병 환자에게 ABO혈액형과 당뇨병 관계를 입증한 연구 자료를 바탕으로 혈색소증(철결핍), 선천적인 질병 등의 이유로 헌혈할 수 없었던


In [12]:
# 정보와 QA 샘플을 주고 질문한 경우
gen("""
다음 내용을 보고 질문에 답변해줘

누654나(1)주 항체 결합력검사의 급여기준
누654나(1)주. 항체 결합력 검사는 각 검사항목의 특이 IgM 항체가 양성 또는 특이 IgM 항체가 음성이지만 IgG 항체가 증가되어 급성기감염이 의심되는 경우에 인정함.
※ 보건복지부 고시 제2021-183호(2021.7.1. 시행)
■ 개정 사유: 문구변경

Q:Rubella 항체 결합력 검사의 급여 기준이 변경된 이유는 무엇인가요?
A:Rubella 항체 결합력 검사의 급여 기준이 변경된 이유는 건강보험 보장성 강화 정책과 관련이 있습니다.
Q:Rubella 항체 결합력 검사의 급여 기준은 어떤 고시에 근거하고 있나요?
A:Rubella 항체 결합력 검사의 급여 기준은 고시 제2018-190호에 근거하고 있습니다.
Q:Rubella 항체 결합력 검사에서 특이 IgM 항체와 IgG 항체의 양성 여부는 어떻게 판단되나요?
A:Rubella 항체 결합력 검사에서는 특이 IgM 항체가 양성이거나 회복기에 특이 IgG 항체가 증가한 경우에 판단됩니다.
Q:Rubella 항체 결합력 검사의 신설된 급여 기준은 언제부터 시행되고 있나요?
A:Rubella 항체 결합력 검사의 신설된 급여 기준은 '18.10.1.부터 시행되고 있습니다.
Q:Rubella 항체 결합력 검사의 급여 기준이 건강보험 보장성 강화 정책과 어떻게 관련되어 있는가요?
A:Rubella 항체 결합력 검사의 급여 기준이 변경된 이유 중 하나는 건강보험 보장성 강화 정책에 부합하기 위함입니다.
Q:Rubella 항체 결합력 검사의 급여 기준은 무엇인가요?
""")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 
다음 내용을 보고 질문에 답변해줘

누654나(1)주 항체 결합력검사의 급여기준
누654나(1)주. 항체 결합력 검사는 각 검사항목의 특이 IgM 항체가 양성 또는 특이 IgM 항체가 음성이지만 IgG 항체가 증가되어 급성기감염이 의심되는 경우에 인정함.
※ 보건복지부 고시 제2021-183호(2021.7.1. 시행)
■ 개정 사유: 문구변경

Q:Rubella 항체 결합력 검사의 급여 기준이 변경된 이유는 무엇인가요?	
A:Rubella 항체 결합력 검사의 급여 기준이 변경된 이유는 건강보험 보장성 강화 정책과 관련이 있습니다.
Q:Rubella 항체 결합력 검사의 급여 기준은 어떤 고시에 근거하고 있나요?	
A:Rubella 항체 결합력 검사의 급여 기준은 고시 제2018-190호에 근거하고 있습니다.
Q:Rubella 항체 결합력 검사에서 특이 IgM 항체와 IgG 항체의 양성 여부는 어떻게 판단되나요?	
A:Rubella 항체 결합력 검사에서는 특이 IgM 항체가 양성이거나 회복기에 특이 IgG 항체가 증가한 경우에 판단됩니다.
Q:Rubella 항체 결합력 검사의 신설된 급여 기준은 언제부터 시행되고 있나요?	
A:Rubella 항체 결합력 검사의 신설된 급여 기준은 '18.10.1.부터 시행되고 있습니다.
Q:Rubella 항체 결합력 검사의 급여 기준이 건강보험 보장성 강화 정책과 어떻게 관련되어 있는가요?	
A:Rubella 항체 결합력 검사의 급여 기준이 변경된 이유 중 하나는 건강보험 보장성 강화 정책에 부합하기 위함입니다. 
Q:Rubella 항체 결합력 검사의 급여 기준은 무엇인가요?


### 답변: 
Rubella 항체 결합력 검사는 특이 IgM 항체가 양성이지만 IgG 항체가 증가하여 급성기 감염이 의심되는 경우에 급여가 인정됩니다.

즉, Rubella 항체 결합력 검사의 급여 기준은 특이 IgM 항체와 특이 IgG 항체가 양성이지만, 항체가 증가되어 급성기 감염이 의심되는 경우에 해당됩니다.
검사결과에서 특이 Ig

In [13]:
# 질문만한 경우
gen("""
임신으로 인한 철결핍성 빈혈의 경우 액제형 철분제제의 요양급여가 인정되나요?
""")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 
임신으로 인한 철결핍성 빈혈의 경우 액제형 철분제제의 요양급여가 인정되나요?


### 답변: 철결핍성 빈혈은 일반적으로 비경구수액제로 철분제를 충분한 용량으로 주사하는 것이 추천됩니다. 임신으로 다량의 철분이 사용된 경우는 액제형 철분제제의 경우 사용을 인정하는 건강보험 기준은 없습니다. 그러나, 철분제를 추가하는 것이 필요한 경우가 있습니다. 이런 경우에는 의사가 판단하여 추가하는 경우에 한하여 건강보험급여가 가능합니다. 따라서, 임신 중 건강관리를 위해서는 충분한 상담을 하셔야합니다. 
추가적인 질문이 있으신 경우는 언제든지 연락주시면 상세히 답변드리겠습니다. 

※ 추가적인 질문은 추가적인 질문 게시판과 카톡에서 해주시면 답변드리겠습니다. �### 답변: 
, 
추가적인 답변이나 상담이 필요하신 경우, 아래의 연락처나 인터넷 상담 서비스를 이용하시면 신속히 도움을 드리겠습니다. (인터넷 상담: www.hira.or.kr>인터넷상담>병원예약>추가상담)  
1. 인터넷 상담: 인터넷 상담 신청은 전화로는 연결되지 않습니다. 인터넷 상담


In [14]:
# 정보를 제공하고 질문한 경우
gen("""
다음 내용을 보고 질문에 답변해줘

■ 고시 개정 전체내용
허가사항 범위 내에서 아래와 같은 기준으로 투여 시 요양급여를 인정하며, 동 인정기준 이외에는 약값 전액을 환자가 부담토록 함.
- 아 래 -
가. 일반적인 철결핍성 빈혈의 경우
혈액검사결과 다음에 해당되고 타 경구 철분제제 투여 시 위장장애가 있는 경우에 급여하며, 투여기간은 4～6개월 급여함.
- 다 음 -
1) 일반환자
혈청 페리틴(Ferritin) 30ng/mL 미만 또는 트랜스페린포화도(Transferrin saturation) 20% 미만인 경우
2) 만성신부전증 환자 및 항암화학요법을 받고 있는 비골수성 악성종양을 가진 환자
혈청 페리틴(Ferritin) 100ng/mL 미만 또는 트랜스페린포화도(Transferrin saturation) 20% 미만인 경우
나. 임신으로 인한 철결핍성 빈혈의 경우
혈액검사결과 헤모글로빈(Hb) 11g/dL이하이고, 타 경구 철분제제 투여 시 위장장애가 있는 경우에 급여하며, 투여기간은 4～6개월로 함.
다. 급성출혈 등으로 인한 산후 빈혈의 경우
혈액검사결과 Hb 10g/dL이하인 경우에 급여하며, 투여기간은 4주로 함.
라. 8세 미만의 소아는 철결핍성 빈혈이 확인된 경우
1차로 투여 시에도 요양급여하며, 미숙아의 경우는 예방 투여 시에도 인정함.
■ 고시 개정 고시번호(시행일자)
고시 제2020-79호(2020.5.8.)
■ 고시 개정 사유
철분주사제 급여기준 확대에 따른 교과서, 제외국 가이드라인, 임상문헌, 학회의견 등 참조하여 Hb 수치, 혈청 페리틴(serum ferritin) 및 트랜스페린 포화도(transferrin saturation)의 기준을 완화하여 급여를 확대함

임신으로 인한 철결핍성 빈혈의 경우 액제형 철분제제의 요양급여가 인정되나요?
""")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 
다음 내용을 보고 질문에 답변해줘

■ 고시 개정 전체내용
허가사항 범위 내에서 아래와 같은 기준으로 투여 시 요양급여를 인정하며, 동 인정기준 이외에는 약값 전액을 환자가 부담토록 함.
- 아 래 -
가. 일반적인 철결핍성 빈혈의 경우
혈액검사결과 다음에 해당되고 타 경구 철분제제 투여 시 위장장애가 있는 경우에 급여하며, 투여기간은 4～6개월 급여함.
- 다 음 -
1) 일반환자
혈청 페리틴(Ferritin) 30ng/mL 미만 또는 트랜스페린포화도(Transferrin saturation) 20% 미만인 경우
2) 만성신부전증 환자 및 항암화학요법을 받고 있는 비골수성 악성종양을 가진 환자
혈청 페리틴(Ferritin) 100ng/mL 미만 또는 트랜스페린포화도(Transferrin saturation) 20% 미만인 경우
나. 임신으로 인한 철결핍성 빈혈의 경우
혈액검사결과 헤모글로빈(Hb) 11g/dL이하이고, 타 경구 철분제제 투여 시 위장장애가 있는 경우에 급여하며, 투여기간은 4～6개월로 함.
다. 급성출혈 등으로 인한 산후 빈혈의 경우
혈액검사결과 Hb 10g/dL이하인 경우에 급여하며, 투여기간은 4주로 함.
라. 8세 미만의 소아는 철결핍성 빈혈이 확인된 경우
1차로 투여 시에도 요양급여하며, 미숙아의 경우는 예방 투여 시에도 인정함.
■ 고시 개정 고시번호(시행일자)
고시 제2020-79호(2020.5.8.)
■ 고시 개정 사유
철분주사제 급여기준 확대에 따른 교과서, 제외국 가이드라인, 임상문헌, 학회의견 등 참조하여 Hb 수치, 혈청 페리틴(serum ferritin) 및 트랜스페린 포화도(transferrin saturation)의 기준을 완화하여 급여를 확대함

임신으로 인한 철결핍성 빈혈의 경우 액제형 철분제제의 요양급여가 인정되나요?


### 답변: �원래 철분제제의 허가사항 범위 내에서는 철분결핍성 빈혈의 경우 일반적인 경구 철분제제를 인정하고 있었으나, 최근 철분 결핍환자가 늘어남에 따라 임신으로 인한 철결

In [15]:
# 정보와 QA 샘플을 주고 질문한 경우
gen("""
다음 내용을 보고 질문에 답변해줘

■ 고시 개정 전체내용
허가사항 범위 내에서 아래와 같은 기준으로 투여 시 요양급여를 인정하며, 동 인정기준 이외에는 약값 전액을 환자가 부담토록 함.
- 아 래 -
가. 일반적인 철결핍성 빈혈의 경우
혈액검사결과 다음에 해당되고 타 경구 철분제제 투여 시 위장장애가 있는 경우에 급여하며, 투여기간은 4～6개월 급여함.
- 다 음 -
1) 일반환자
혈청 페리틴(Ferritin) 30ng/mL 미만 또는 트랜스페린포화도(Transferrin saturation) 20% 미만인 경우
2) 만성신부전증 환자 및 항암화학요법을 받고 있는 비골수성 악성종양을 가진 환자
혈청 페리틴(Ferritin) 100ng/mL 미만 또는 트랜스페린포화도(Transferrin saturation) 20% 미만인 경우
나. 임신으로 인한 철결핍성 빈혈의 경우
혈액검사결과 헤모글로빈(Hb) 11g/dL이하이고, 타 경구 철분제제 투여 시 위장장애가 있는 경우에 급여하며, 투여기간은 4～6개월로 함.
다. 급성출혈 등으로 인한 산후 빈혈의 경우
혈액검사결과 Hb 10g/dL이하인 경우에 급여하며, 투여기간은 4주로 함.
라. 8세 미만의 소아는 철결핍성 빈혈이 확인된 경우
1차로 투여 시에도 요양급여하며, 미숙아의 경우는 예방 투여 시에도 인정함.
■ 고시 개정 고시번호(시행일자)
고시 제2020-79호(2020.5.8.)
■ 고시 개정 사유
철분주사제 급여기준 확대에 따른 교과서, 제외국 가이드라인, 임상문헌, 학회의견 등 참조하여 Hb 수치, 혈청 페리틴(serum ferritin) 및 트랜스페린 포화도(transferrin saturation)의 기준을 완화하여 급여를 확대함

Q:액제형 철분제제의 급여 확대는 어떤 근거에 기반하고 있나요?
A:액제형 철분제제의 급여 확대는 철분주사제 급여기준 확대에 따른 교과서, 제외국 가이드라인, 임상문헌, 학회의견 등을 참조하여 이루어졌습니다
Q:액제형 철분제제의 급여 확대로 어떤 부분이 변경되었나요?
A:Hb 수치, 혈청 페리틴, 트랜스페린 포화도의 기준이 완화되어 급여 대상이 확대되었습니다.
Q:액제형 철분제제의 급여 확대에 참고된 주요 문헌은 어떤 것들이 있나요?
A:액제형 철분제제의 급여 확대에는 Goldman-Cecil Medicine, Gahart’s Intravenous Medications, WHO, NICE guideline, NATA guidelines, Cochrane Library 등의 주요 문헌이 참고되었습니다.
Q:액제형 철분제제의 허가사항 범위 내에서 투여 시 어떤 경우에 요양급여가 인정되나요?
A:액제형 철분제제의 허가사항 범위 내에서 요양급여가 인정되는 경우는 일반적인 철결핍성 빈혈, 임신으로 인한 철결핍성 빈혈, 급성 출혈 등으로 인한 산후 빈혈, 그리고 8세 미만의 소아에서 철결핍성 빈혈이 확인된 경우입니다
Q:액제형 철분제제를 투여받기 위한 혈액검사 결과의 주요 기준은 무엇인가요?
A:액제형 철분제제를 투여받기 위한 주요 혈액검사 결과 기준은 혈청 페리틴이 30ng/mL 미만이거나 트랜스페린포화도가 20% 미만인 경우입니다.
Q:액제형 철분제제의 급여기간은 어떻게 결정되나요?
A:액제형 철분제제의 급여기간은 투여 대상에 따라 다르며, 일반적인 철결핍성 빈혈의 경우 4~6개월 동안 급여되며, 산후 빈혈의 경우는 4주 동안 급여됩니다.
Q:액제형 철분제제를 투여받는데 있어 요양급여가 적용되지 않는 경우는 어떤 경우인가요?
A:액제형 철분제제를 투여받는데 있어서 요양급여가 인정되지 않는 경우는 약값이 허가사항 범위 이외에 있는 경우입니다.
Q:액제형 철분제제의 급여 인정 기준 외에 부담하는 약값은 어떤 경우인가요?
A:액제형 철분제제의 급여 인정 기준 외에는 환자가 약값을 전액 부담해야 합니다.
Q:액제형 철분제제의 요양급여 인정 기준 외의 경우에는 어떤 현상이 나타날 수 있나요?
A:액제형 철분제제의 요양급여 인정 기준 외의 경우에는 환자가 약값을 전액 부담하게 되며, 급여 혜택을 받을 수 없습니다
Q:액제형 철분제제의 급여 확대는 어떤 이유로 이루어졌나요?
A:액제형 철분제제의 급여 확대는 철분주사제 급여기준 확대에 따른 교과서, 제외국 가이드라인, 임상문헌, 학회의견 등을 참조하여 이루어졌습니다
Q:임신으로 인한 철결핍성 빈혈의 경우 액제형 철분제제의 요양급여가 인정되나요?
""")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문:  
다음 내용을 보고 질문에 답변해줘

■ 고시 개정 전체내용
허가사항 범위 내에서 아래와 같은 기준으로 투여 시 요양급여를 인정하며, 동 인정기준 이외에는 약값 전액을 환자가 부담토록 함.
- 아 래 -
가. 일반적인 철결핍성 빈혈의 경우
혈액검사결과 다음에 해당되고 타 경구 철분제제 투여 시 위장장애가 있는 경우에 급여하며, 투여기간은 4～6개월 급여함.
- 다 음 -
1) 일반환자
혈청 페리틴(Ferritin) 30ng/mL 미만 또는 트랜스페린포화도(Transferrin saturation) 20% 미만인 경우
2) 만성신부전증 환자 및 항암화학요법을 받고 있는 비골수성 악성종양을 가진 환자
혈청 페리틴(Ferritin) 100ng/mL 미만 또는 트랜스페린포화도(Transferrin saturation) 20% 미만인 경우
나. 임신으로 인한 철결핍성 빈혈의 경우
혈액검사결과 헤모글로빈(Hb) 11g/dL이하이고, 타 경구 철분제제 투여 시 위장장애가 있는 경우에 급여하며, 투여기간은 4～6개월로 함.
다. 급성출혈 등으로 인한 산후 빈혈의 경우
혈액검사결과 Hb 10g/dL이하인 경우에 급여하며, 투여기간은 4주로 함.
라. 8세 미만의 소아는 철결핍성 빈혈이 확인된 경우
1차로 투여 시에도 요양급여하며, 미숙아의 경우는 예방 투여 시에도 인정함.
■ 고시 개정 고시번호(시행일자)
고시 제2020-79호(2020.5.8.)
■ 고시 개정 사유
철분주사제 급여기준 확대에 따른 교과서, 제외국 가이드라인, 임상문헌, 학회의견 등 참조하여 Hb 수치, 혈청 페리틴(serum ferritin) 및 트랜스페린 포화도(transferrin saturation)의 기준을 완화하여 급여를 확대함

Q:액제형 철분제제의 급여 확대는 어떤 근거에 기반하고 있나요?	
A:액제형 철분제제의 급여 확대는 철분주사제 급여기준 확대에 따른 교과서, 제외국 가이드라인, 임상문헌, 학회의견 등을 참조하여 이루어졌습니다
Q:액제형 철분제제의 급여 확대로 어떤 부분이 

In [16]:
# 정보와 QA 샘플을 주고 질문한 경우
gen("""
다음 내용을 보고 질문에 답변해줘

■ 고시 개정 전체내용
허가사항 범위 내에서 아래와 같은 기준으로 투여 시 요양급여를 인정하며, 동 인정기준 이외에는 약값 전액을 환자가 부담토록 함.
- 아 래 -
가. 일반적인 철결핍성 빈혈의 경우
혈액검사결과 다음에 해당되고 타 경구 철분제제 투여 시 위장장애가 있는 경우에 급여하며, 투여기간은 4～6개월 급여함.
- 다 음 -
1) 일반환자
혈청 페리틴(Ferritin) 30ng/mL 미만 또는 트랜스페린포화도(Transferrin saturation) 20% 미만인 경우
2) 만성신부전증 환자 및 항암화학요법을 받고 있는 비골수성 악성종양을 가진 환자
혈청 페리틴(Ferritin) 100ng/mL 미만 또는 트랜스페린포화도(Transferrin saturation) 20% 미만인 경우
나. 임신으로 인한 철결핍성 빈혈의 경우
혈액검사결과 헤모글로빈(Hb) 11g/dL이하이고, 타 경구 철분제제 투여 시 위장장애가 있는 경우에 급여하며, 투여기간은 4～6개월로 함.
다. 급성출혈 등으로 인한 산후 빈혈의 경우
혈액검사결과 Hb 10g/dL이하인 경우에 급여하며, 투여기간은 4주로 함.
라. 8세 미만의 소아는 철결핍성 빈혈이 확인된 경우
1차로 투여 시에도 요양급여하며, 미숙아의 경우는 예방 투여 시에도 인정함.
■ 고시 개정 고시번호(시행일자)
고시 제2020-79호(2020.5.8.)
■ 고시 개정 사유
철분주사제 급여기준 확대에 따른 교과서, 제외국 가이드라인, 임상문헌, 학회의견 등 참조하여 Hb 수치, 혈청 페리틴(serum ferritin) 및 트랜스페린 포화도(transferrin saturation)의 기준을 완화하여 급여를 확대함

Q:액제형 철분제제의 급여 확대는 어떤 근거에 기반하고 있나요?
A:액제형 철분제제의 급여 확대는 철분주사제 급여기준 확대에 따른 교과서, 제외국 가이드라인, 임상문헌, 학회의견 등을 참조하여 이루어졌습니다
Q:액제형 철분제제의 급여 확대로 어떤 부분이 변경되었나요?
A:Hb 수치, 혈청 페리틴, 트랜스페린 포화도의 기준이 완화되어 급여 대상이 확대되었습니다.
Q:액제형 철분제제의 급여 확대에 참고된 주요 문헌은 어떤 것들이 있나요?
A:액제형 철분제제의 급여 확대에는 Goldman-Cecil Medicine, Gahart’s Intravenous Medications, WHO, NICE guideline, NATA guidelines, Cochrane Library 등의 주요 문헌이 참고되었습니다.
Q:액제형 철분제제의 허가사항 범위 내에서 투여 시 어떤 경우에 요양급여가 인정되나요?
A:액제형 철분제제의 허가사항 범위 내에서 요양급여가 인정되는 경우는 일반적인 철결핍성 빈혈, 임신으로 인한 철결핍성 빈혈, 급성 출혈 등으로 인한 산후 빈혈, 그리고 8세 미만의 소아에서 철결핍성 빈혈이 확인된 경우입니다
Q:액제형 철분제제를 투여받기 위한 혈액검사 결과의 주요 기준은 무엇인가요?
A:액제형 철분제제를 투여받기 위한 주요 혈액검사 결과 기준은 혈청 페리틴이 30ng/mL 미만이거나 트랜스페린포화도가 20% 미만인 경우입니다.
Q:액제형 철분제제의 급여기간은 어떻게 결정되나요?
A:액제형 철분제제의 급여기간은 투여 대상에 따라 다르며, 일반적인 철결핍성 빈혈의 경우 4~6개월 동안 급여되며, 산후 빈혈의 경우는 4주 동안 급여됩니다.
Q:액제형 철분제제를 투여받는데 있어 요양급여가 적용되지 않는 경우는 어떤 경우인가요?
A:액제형 철분제제를 투여받는데 있어서 요양급여가 인정되지 않는 경우는 약값이 허가사항 범위 이외에 있는 경우입니다.
Q:액제형 철분제제의 급여 인정 기준 외에 부담하는 약값은 어떤 경우인가요?
A:액제형 철분제제의 급여 인정 기준 외에는 환자가 약값을 전액 부담해야 합니다.
Q:액제형 철분제제의 요양급여 인정 기준 외의 경우에는 어떤 현상이 나타날 수 있나요?
A:액제형 철분제제의 요양급여 인정 기준 외의 경우에는 환자가 약값을 전액 부담하게 되며, 급여 혜택을 받을 수 없습니다
Q:액제형 철분제제의 급여 확대는 어떤 이유로 이루어졌나요?
A:액제형 철분제제의 급여 확대는 철분주사제 급여기준 확대에 따른 교과서, 제외국 가이드라인, 임상문헌, 학회의견 등을 참조하여 이루어졌습니다
Q:임신으로 인한 철결핍성 빈혈의 경우 혈액검사결과 Hb 수치가 얼마일때 급여로 인정되나요?
""")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문:  
다음 내용을 보고 질문에 답변해줘

■ 고시 개정 전체내용
허가사항 범위 내에서 아래와 같은 기준으로 투여 시 요양급여를 인정하며, 동 인정기준 이외에는 약값 전액을 환자가 부담토록 함.
- 아 래 -
가. 일반적인 철결핍성 빈혈의 경우
혈액검사결과 다음에 해당되고 타 경구 철분제제 투여 시 위장장애가 있는 경우에 급여하며, 투여기간은 4～6개월 급여함.
- 다 음 -
1) 일반환자
혈청 페리틴(Ferritin) 30ng/mL 미만 또는 트랜스페린포화도(Transferrin saturation) 20% 미만인 경우
2) 만성신부전증 환자 및 항암화학요법을 받고 있는 비골수성 악성종양을 가진 환자
혈청 페리틴(Ferritin) 100ng/mL 미만 또는 트랜스페린포화도(Transferrin saturation) 20% 미만인 경우
나. 임신으로 인한 철결핍성 빈혈의 경우
혈액검사결과 헤모글로빈(Hb) 11g/dL이하이고, 타 경구 철분제제 투여 시 위장장애가 있는 경우에 급여하며, 투여기간은 4～6개월로 함.
다. 급성출혈 등으로 인한 산후 빈혈의 경우
혈액검사결과 Hb 10g/dL이하인 경우에 급여하며, 투여기간은 4주로 함.
라. 8세 미만의 소아는 철결핍성 빈혈이 확인된 경우
1차로 투여 시에도 요양급여하며, 미숙아의 경우는 예방 투여 시에도 인정함.
■ 고시 개정 고시번호(시행일자)
고시 제2020-79호(2020.5.8.)
■ 고시 개정 사유
철분주사제 급여기준 확대에 따른 교과서, 제외국 가이드라인, 임상문헌, 학회의견 등 참조하여 Hb 수치, 혈청 페리틴(serum ferritin) 및 트랜스페린 포화도(transferrin saturation)의 기준을 완화하여 급여를 확대함

Q:액제형 철분제제의 급여 확대는 어떤 근거에 기반하고 있나요?	
A:액제형 철분제제의 급여 확대는 철분주사제 급여기준 확대에 따른 교과서, 제외국 가이드라인, 임상문헌, 학회의견 등을 참조하여 이루어졌습니다
Q:액제형 철분제제의 급여 확대로 어떤 부분이 

In [17]:
# 정보와 QA 샘플을 주고 질문한 경우
gen("""
다음 내용을 보고 질문에 답변해줘

■ 고시 개정 전체내용
허가사항 범위 내에서 아래와 같은 기준으로 투여 시 요양급여를 인정하며, 동 인정기준 이외에는 약값 전액을 환자가 부담토록 함.
- 아 래 -
가. 일반적인 철결핍성 빈혈의 경우
혈액검사결과 다음에 해당되고 타 경구 철분제제 투여 시 위장장애가 있는 경우에 급여하며, 투여기간은 4～6개월 급여함.
- 다 음 -
1) 일반환자
혈청 페리틴(Ferritin) 30ng/mL 미만 또는 트랜스페린포화도(Transferrin saturation) 20% 미만인 경우
2) 만성신부전증 환자 및 항암화학요법을 받고 있는 비골수성 악성종양을 가진 환자
혈청 페리틴(Ferritin) 100ng/mL 미만 또는 트랜스페린포화도(Transferrin saturation) 20% 미만인 경우
나. 임신으로 인한 철결핍성 빈혈의 경우
혈액검사결과 헤모글로빈(Hb) 11g/dL이하이고, 타 경구 철분제제 투여 시 위장장애가 있는 경우에 급여하며, 투여기간은 4～6개월로 함.
다. 급성출혈 등으로 인한 산후 빈혈의 경우
혈액검사결과 Hb 10g/dL이하인 경우에 급여하며, 투여기간은 4주로 함.

Q:액제형 철분제제의 급여 확대는 어떤 근거에 기반하고 있나요?
A:액제형 철분제제의 급여 확대는 철분주사제 급여기준 확대에 따른 교과서, 제외국 가이드라인, 임상문헌, 학회의견 등을 참조하여 이루어졌습니다
Q:액제형 철분제제의 급여 확대로 어떤 부분이 변경되었나요?
A:Hb 수치, 혈청 페리틴, 트랜스페린 포화도의 기준이 완화되어 급여 대상이 확대되었습니다.
Q:액제형 철분제제의 급여 확대에 참고된 주요 문헌은 어떤 것들이 있나요?
A:액제형 철분제제의 급여 확대에는 Goldman-Cecil Medicine, Gahart’s Intravenous Medications, WHO, NICE guideline, NATA guidelines, Cochrane Library 등의 주요 문헌이 참고되었습니다.
Q:액제형 철분제제의 허가사항 범위 내에서 투여 시 어떤 경우에 요양급여가 인정되나요?
A:액제형 철분제제의 허가사항 범위 내에서 요양급여가 인정되는 경우는 일반적인 철결핍성 빈혈, 임신으로 인한 철결핍성 빈혈, 급성 출혈 등으로 인한 산후 빈혈, 그리고 8세 미만의 소아에서 철결핍성 빈혈이 확인된 경우입니다
Q:액제형 철분제제를 투여받기 위한 혈액검사 결과의 주요 기준은 무엇인가요?
A:액제형 철분제제를 투여받기 위한 주요 혈액검사 결과 기준은 혈청 페리틴이 30ng/mL 미만이거나 트랜스페린포화도가 20% 미만인 경우입니다.
Q:액제형 철분제제의 급여기간은 어떻게 결정되나요?
A:액제형 철분제제의 급여기간은 투여 대상에 따라 다르며, 일반적인 철결핍성 빈혈의 경우 4~6개월 동안 급여되며, 산후 빈혈의 경우는 4주 동안 급여됩니다.
Q:액제형 철분제제를 투여받는데 있어 요양급여가 적용되지 않는 경우는 어떤 경우인가요?
A:액제형 철분제제를 투여받는데 있어서 요양급여가 인정되지 않는 경우는 약값이 허가사항 범위 이외에 있는 경우입니다.
Q:액제형 철분제제의 급여 인정 기준 외에 부담하는 약값은 어떤 경우인가요?
A:액제형 철분제제의 급여 인정 기준 외에는 환자가 약값을 전액 부담해야 합니다.
Q:액제형 철분제제의 요양급여 인정 기준 외의 경우에는 어떤 현상이 나타날 수 있나요?
A:액제형 철분제제의 요양급여 인정 기준 외의 경우에는 환자가 약값을 전액 부담하게 되며, 급여 혜택을 받을 수 없습니다
Q:액제형 철분제제의 급여 확대는 어떤 이유로 이루어졌나요?
A:액제형 철분제제의 급여 확대는 철분주사제 급여기준 확대에 따른 교과서, 제외국 가이드라인, 임상문헌, 학회의견 등을 참조하여 이루어졌습니다
Q:임신으로 인한 철결핍성 빈혈의 경우 혈액검사결과 헤모글로빈 수치가 얼마일때 급여로 인정되나요?
""")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문:  
다음 내용을 보고 질문에 답변해줘

■ 고시 개정 전체내용
허가사항 범위 내에서 아래와 같은 기준으로 투여 시 요양급여를 인정하며, 동 인정기준 이외에는 약값 전액을 환자가 부담토록 함.
- 아 래 -
가. 일반적인 철결핍성 빈혈의 경우
혈액검사결과 다음에 해당되고 타 경구 철분제제 투여 시 위장장애가 있는 경우에 급여하며, 투여기간은 4～6개월 급여함.
- 다 음 -
1) 일반환자
혈청 페리틴(Ferritin) 30ng/mL 미만 또는 트랜스페린포화도(Transferrin saturation) 20% 미만인 경우
2) 만성신부전증 환자 및 항암화학요법을 받고 있는 비골수성 악성종양을 가진 환자
혈청 페리틴(Ferritin) 100ng/mL 미만 또는 트랜스페린포화도(Transferrin saturation) 20% 미만인 경우
나. 임신으로 인한 철결핍성 빈혈의 경우
혈액검사결과 헤모글로빈(Hb) 11g/dL이하이고, 타 경구 철분제제 투여 시 위장장애가 있는 경우에 급여하며, 투여기간은 4～6개월로 함.
다. 급성출혈 등으로 인한 산후 빈혈의 경우
혈액검사결과 Hb 10g/dL이하인 경우에 급여하며, 투여기간은 4주로 함.

Q:액제형 철분제제의 급여 확대는 어떤 근거에 기반하고 있나요?	
A:액제형 철분제제의 급여 확대는 철분주사제 급여기준 확대에 따른 교과서, 제외국 가이드라인, 임상문헌, 학회의견 등을 참조하여 이루어졌습니다
Q:액제형 철분제제의 급여 확대로 어떤 부분이 변경되었나요?	
A:Hb 수치, 혈청 페리틴, 트랜스페린 포화도의 기준이 완화되어 급여 대상이 확대되었습니다.
Q:액제형 철분제제의 급여 확대에 참고된 주요 문헌은 어떤 것들이 있나요?	
A:액제형 철분제제의 급여 확대에는 Goldman-Cecil Medicine, Gahart’s Intravenous Medications, WHO, NICE guideline, NATA guidelines, Cochrane Library 등의 주요 문헌이 참고되었습니다.
Q:액제형 

In [21]:
gen("""
내가 제공한 정보를 보고 질문에 간단하게 답변해줘

나732-2 태아심음자궁수축검사의 인정기준
나732-2 태아심음자궁수축검사는 한 시간 당 8회 이상의 자궁수축이 느껴지거나 조기진통이 의심되는 임신 37주 이전 산모에게 시행하는 경우에 요양급여를 인정함.

Q나732-2 태아심음자궁수축검사의 고시 신설/개정 고시번호와 시행일은 어떻게 되나요?
A나732-2 태아심음자궁수축검사의 고시 신설/개정 고시번호는 제2023-56호이며, 시행일은 2023년 3월 29일입니다.
Q나732-2 태아심음자궁수축검사의 변경 전 고시번호와 시행일은 어떤 것이었나요?
A변경 전 고시번호는 제2021-21호이며, 시행일은 2021년 2월 1일이었습니다.
Q나732-2 태아심음자궁수축검사의 변경 전 고시내용에 어떤 수정이 있었나요?
A2022년 심사기준 간 정합성 정비에 따른 문구 수정으로, 내용에는 문구 수정이 있었습니다.
Q나732-2 태아심음자궁수축검사의 요양급여 인정 기준은 언제 변경되었나요?
A요양급여 인정 기준은 2023년 3월 29일에 제2023-56호로 신설/개정되었습니다.
Q나732-2 태아심음자궁수축검사의 고시를 신설/개정한 이유는 무엇이었나요?
A2022년 심사기준 간 정합성 정비에 따른 문구 수정을 위해 신설/개정되었습니다.
Q나732-2 태아심음자궁수축검사의 변경 전 고시와 변경 후 고시 간에 어떤 차이가 있나요?
A변경 전 고시에서는 조기진통이 의심되는 경우에 시행되는 것으로 명시되어 있었으나, 변경 후 고시에서는 자궁수축이 8회 이상 느껴지거나 조기진통이 의심되는 경우에 요양급여를 인정하도록 수정되었습니다.
Q나732-2 태아심음자궁수축검사의 고시에 따라 요양급여가 인정되는 산모의 임신 기간은 어떻게 되나요?
A요양급여가 인정되는 산모의 임신 기간은 37주 이전입니다.
Q:나732-2 태아심음자궁수축검사의 요양급여 인정 기준은 무엇인가요?
""")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 
내가 제공한 정보를 보고 질문에 간단하게 답변해줘

나732-2 태아심음자궁수축검사의 인정기준
나732-2 태아심음자궁수축검사는 한 시간 당 8회 이상의 자궁수축이 느껴지거나 조기진통이 의심되는 임신 37주 이전 산모에게 시행하는 경우에 요양급여를 인정함.

Q나732-2 태아심음자궁수축검사의 고시 신설/개정 고시번호와 시행일은 어떻게 되나요?	
A나732-2 태아심음자궁수축검사의 고시 신설/개정 고시번호는 제2023-56호이며, 시행일은 2023년 3월 29일입니다.
Q나732-2 태아심음자궁수축검사의 변경 전 고시번호와 시행일은 어떤 것이었나요?	
A변경 전 고시번호는 제2021-21호이며, 시행일은 2021년 2월 1일이었습니다.
Q나732-2 태아심음자궁수축검사의 변경 전 고시내용에 어떤 수정이 있었나요?	
A2022년 심사기준 간 정합성 정비에 따른 문구 수정으로, 내용에는 문구 수정이 있었습니다.
Q나732-2 태아심음자궁수축검사의 요양급여 인정 기준은 언제 변경되었나요?	
A요양급여 인정 기준은 2023년 3월 29일에 제2023-56호로 신설/개정되었습니다.
Q나732-2 태아심음자궁수축검사의 고시를 신설/개정한 이유는 무엇이었나요?	
A2022년 심사기준 간 정합성 정비에 따른 문구 수정을 위해 신설/개정되었습니다.
Q나732-2 태아심음자궁수축검사의 변경 전 고시와 변경 후 고시 간에 어떤 차이가 있나요?	
A변경 전 고시에서는 조기진통이 의심되는 경우에 시행되는 것으로 명시되어 있었으나, 변경 후 고시에서는 자궁수축이 8회 이상 느껴지거나 조기진통이 의심되는 경우에 요양급여를 인정하도록 수정되었습니다.
Q나732-2 태아심음자궁수축검사의 고시에 따라 요양급여가 인정되는 산모의 임신 기간은 어떻게 되나요?	
A요양급여가 인정되는 산모의 임신 기간은 37주 이전입니다.
Q:나732-2 태아심음자궁수축검사의 요양급여 인정 기준은 무엇인가요?


### 답변: 
제가 제공한 정보를 보고 질문에 간단하게 답변해줘
제732-2 태아심음

In [23]:
# 정보와 QA 샘플을 주고 질문한 경우 Max_new_Tokens을 500으로 변경
gen("""
다음 내용을 보고 질문에 답변해줘

■ 고시 개정 전체내용
허가사항 범위 내에서 아래와 같은 기준으로 투여 시 요양급여를 인정하며, 동 인정기준 이외에는 약값 전액을 환자가 부담토록 함.
- 아 래 -
가. 일반적인 철결핍성 빈혈의 경우
혈액검사결과 다음에 해당되고 타 경구 철분제제 투여 시 위장장애가 있는 경우에 급여하며, 투여기간은 4～6개월 급여함.
- 다 음 -
1) 일반환자
혈청 페리틴(Ferritin) 30ng/mL 미만 또는 트랜스페린포화도(Transferrin saturation) 20% 미만인 경우
2) 만성신부전증 환자 및 항암화학요법을 받고 있는 비골수성 악성종양을 가진 환자
혈청 페리틴(Ferritin) 100ng/mL 미만 또는 트랜스페린포화도(Transferrin saturation) 20% 미만인 경우
나. 임신으로 인한 철결핍성 빈혈의 경우
혈액검사결과 헤모글로빈(Hb) 11g/dL이하이고, 타 경구 철분제제 투여 시 위장장애가 있는 경우에 급여하며, 투여기간은 4～6개월로 함.
다. 급성출혈 등으로 인한 산후 빈혈의 경우
혈액검사결과 Hb 10g/dL이하인 경우에 급여하며, 투여기간은 4주로 함.
라. 8세 미만의 소아는 철결핍성 빈혈이 확인된 경우
1차로 투여 시에도 요양급여하며, 미숙아의 경우는 예방 투여 시에도 인정함.
■ 고시 개정 고시번호(시행일자)
고시 제2020-79호(2020.5.8.)
■ 고시 개정 사유
철분주사제 급여기준 확대에 따른 교과서, 제외국 가이드라인, 임상문헌, 학회의견 등 참조하여 Hb 수치, 혈청 페리틴(serum ferritin) 및 트랜스페린 포화도(transferrin saturation)의 기준을 완화하여 급여를 확대함

Q:액제형 철분제제의 급여 확대는 어떤 근거에 기반하고 있나요?
A:액제형 철분제제의 급여 확대는 철분주사제 급여기준 확대에 따른 교과서, 제외국 가이드라인, 임상문헌, 학회의견 등을 참조하여 이루어졌습니다
Q:액제형 철분제제의 급여 확대로 어떤 부분이 변경되었나요?
A:Hb 수치, 혈청 페리틴, 트랜스페린 포화도의 기준이 완화되어 급여 대상이 확대되었습니다.
Q:액제형 철분제제의 급여 확대에 참고된 주요 문헌은 어떤 것들이 있나요?
A:액제형 철분제제의 급여 확대에는 Goldman-Cecil Medicine, Gahart’s Intravenous Medications, WHO, NICE guideline, NATA guidelines, Cochrane Library 등의 주요 문헌이 참고되었습니다.
Q:액제형 철분제제의 허가사항 범위 내에서 투여 시 어떤 경우에 요양급여가 인정되나요?
A:액제형 철분제제의 허가사항 범위 내에서 요양급여가 인정되는 경우는 일반적인 철결핍성 빈혈, 임신으로 인한 철결핍성 빈혈, 급성 출혈 등으로 인한 산후 빈혈, 그리고 8세 미만의 소아에서 철결핍성 빈혈이 확인된 경우입니다
Q:액제형 철분제제를 투여받기 위한 혈액검사 결과의 주요 기준은 무엇인가요?
A:액제형 철분제제를 투여받기 위한 주요 혈액검사 결과 기준은 혈청 페리틴이 30ng/mL 미만이거나 트랜스페린포화도가 20% 미만인 경우입니다.
Q:액제형 철분제제의 급여기간은 어떻게 결정되나요?
A:액제형 철분제제의 급여기간은 투여 대상에 따라 다르며, 일반적인 철결핍성 빈혈의 경우 4~6개월 동안 급여되며, 산후 빈혈의 경우는 4주 동안 급여됩니다.
Q:액제형 철분제제를 투여받는데 있어 요양급여가 적용되지 않는 경우는 어떤 경우인가요?
A:액제형 철분제제를 투여받는데 있어서 요양급여가 인정되지 않는 경우는 약값이 허가사항 범위 이외에 있는 경우입니다.
Q:액제형 철분제제의 급여 인정 기준 외에 부담하는 약값은 어떤 경우인가요?
A:액제형 철분제제의 급여 인정 기준 외에는 환자가 약값을 전액 부담해야 합니다.
Q:액제형 철분제제의 요양급여 인정 기준 외의 경우에는 어떤 현상이 나타날 수 있나요?
A:액제형 철분제제의 요양급여 인정 기준 외의 경우에는 환자가 약값을 전액 부담하게 되며, 급여 혜택을 받을 수 없습니다
Q:액제형 철분제제의 급여 확대는 어떤 이유로 이루어졌나요?
A:액제형 철분제제의 급여 확대는 철분주사제 급여기준 확대에 따른 교과서, 제외국 가이드라인, 임상문헌, 학회의견 등을 참조하여 이루어졌습니다
Q:임신으로 인한 철결핍성 빈혈의 경우 액제형 철분제제의 요양급여가 인정되나요?
""")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문:  
다음 내용을 보고 질문에 답변해줘

■ 고시 개정 전체내용
허가사항 범위 내에서 아래와 같은 기준으로 투여 시 요양급여를 인정하며, 동 인정기준 이외에는 약값 전액을 환자가 부담토록 함.
- 아 래 -
가. 일반적인 철결핍성 빈혈의 경우
혈액검사결과 다음에 해당되고 타 경구 철분제제 투여 시 위장장애가 있는 경우에 급여하며, 투여기간은 4～6개월 급여함.
- 다 음 -
1) 일반환자
혈청 페리틴(Ferritin) 30ng/mL 미만 또는 트랜스페린포화도(Transferrin saturation) 20% 미만인 경우
2) 만성신부전증 환자 및 항암화학요법을 받고 있는 비골수성 악성종양을 가진 환자
혈청 페리틴(Ferritin) 100ng/mL 미만 또는 트랜스페린포화도(Transferrin saturation) 20% 미만인 경우
나. 임신으로 인한 철결핍성 빈혈의 경우
혈액검사결과 헤모글로빈(Hb) 11g/dL이하이고, 타 경구 철분제제 투여 시 위장장애가 있는 경우에 급여하며, 투여기간은 4～6개월로 함.
다. 급성출혈 등으로 인한 산후 빈혈의 경우
혈액검사결과 Hb 10g/dL이하인 경우에 급여하며, 투여기간은 4주로 함.
라. 8세 미만의 소아는 철결핍성 빈혈이 확인된 경우
1차로 투여 시에도 요양급여하며, 미숙아의 경우는 예방 투여 시에도 인정함.
■ 고시 개정 고시번호(시행일자)
고시 제2020-79호(2020.5.8.)
■ 고시 개정 사유
철분주사제 급여기준 확대에 따른 교과서, 제외국 가이드라인, 임상문헌, 학회의견 등 참조하여 Hb 수치, 혈청 페리틴(serum ferritin) 및 트랜스페린 포화도(transferrin saturation)의 기준을 완화하여 급여를 확대함

Q:액제형 철분제제의 급여 확대는 어떤 근거에 기반하고 있나요?	
A:액제형 철분제제의 급여 확대는 철분주사제 급여기준 확대에 따른 교과서, 제외국 가이드라인, 임상문헌, 학회의견 등을 참조하여 이루어졌습니다
Q:액제형 철분제제의 급여 확대로 어떤 부분이 

In [27]:
def gen(x):
    gened = model.generate(
        **tokenizer(
            f"### 질문: {x}\n\n### 답변:",
            return_tensors='pt',
            return_token_type_ids=False
        ),
        max_new_tokens=500,
        early_stopping=True,
        do_sample=True,
        eos_token_id=2,
    )
    print(tokenizer.decode(gened[0]))
    print('-------------test--------')
    print(tokenizer.decode(gened[0]).split('###')[0])

In [30]:
# 정보와 QA 샘플을 주고 질문한 경우 Max_new_Tokens을 500으로 변경
gen("""
다음 내용을 보고 질문에 답변해줘

허가사항 범위 내에서 아래와 같은 기준으로 투여 시 요양급여를 인정하며, 동 인정기준 이외에는 약값 전액을 환자가 부담토록 함.
- 아 래 -
가. 일반적인 철결핍성 빈혈의 경우
혈액검사결과 다음에 해당되고 타 경구 철분제제 투여 시 위장장애가 있는 경우에 급여하며, 투여기간은 4～6개월 급여함.
- 다 음 -
1) 일반환자
혈청 페리틴(Ferritin) 30ng/mL 미만 또는 트랜스페린포화도(Transferrin saturation) 20% 미만인 경우
2) 만성신부전증 환자 및 항암화학요법을 받고 있는 비골수성 악성종양을 가진 환자
혈청 페리틴(Ferritin) 100ng/mL 미만 또는 트랜스페린포화도(Transferrin saturation) 20% 미만인 경우
나. 임신으로 인한 철결핍성 빈혈의 경우
혈액검사결과 헤모글로빈(Hb) 11g/dL이하이고, 타 경구 철분제제 투여 시 위장장애가 있는 경우에 급여하며, 투여기간은 4～6개월로 함.

Q:액제형 철분제제의 허가사항 범위 내에서 투여 시 어떤 경우에 요양급여가 인정되나요?
A:액제형 철분제제의 허가사항 범위 내에서 요양급여가 인정되는 경우는 일반적인 철결핍성 빈혈, 임신으로 인한 철결핍성 빈혈, 급성 출혈 등으로 인한 산후 빈혈, 그리고 8세 미만의 소아에서 철결핍성 빈혈이 확인된 경우입니다
Q:액제형 철분제제를 투여받기 위한 혈액검사 결과의 주요 기준은 무엇인가요?
A:액제형 철분제제를 투여받기 위한 주요 혈액검사 결과 기준은 혈청 페리틴이 30ng/mL 미만이거나 트랜스페린포화도가 20% 미만인 경우입니다.
Q:액제형 철분제제의 급여기간은 어떻게 결정되나요?
A:액제형 철분제제의 급여기간은 투여 대상에 따라 다르며, 일반적인 철결핍성 빈혈의 경우 4~6개월 동안 급여되며, 산후 빈혈의 경우는 4주 동안 급여됩니다.
Q:액제형 철분제제를 투여받는데 있어 요양급여가 적용되지 않는 경우는 어떤 경우인가요?
A:액제형 철분제제를 투여받는데 있어서 요양급여가 인정되지 않는 경우는 약값이 허가사항 범위 이외에 있는 경우입니다.
Q:액제형 철분제제의 요양급여 인정 기준 외의 경우에는 어떤 현상이 나타날 수 있나요?
A:액제형 철분제제의 요양급여 인정 기준 외의 경우에는 환자가 약값을 전액 부담하게 되며, 급여 혜택을 받을 수 없습니다
Q:액제형 철분제제의 급여 확대는 어떤 이유로 이루어졌나요?
A:액제형 철분제제의 급여 확대는 철분주사제 급여기준 확대에 따른 교과서, 제외국 가이드라인, 임상문헌, 학회의견 등을 참조하여 이루어졌습니다
Q:임신으로 인한 철결핍성 빈혈의 경우 액제형 철분제제의 요양급여가 인정되나요?
""")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


OutOfMemoryError: ignored

In [18]:
# 모델 저장 & 업로드
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' creden

In [19]:
model.push_to_hub('kyounghyun/qlora-koalpaca-polyglot-12.8b-500step')

adapter_model.safetensors:   0%|          | 0.00/26.2M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/kyounghyun/qlora-koalpaca-polyglot-12.8b-500step/commit/d3bdeac7f2d850963268fba0febd9d956696dd5d', commit_message='Upload model', commit_description='', oid='d3bdeac7f2d850963268fba0febd9d956696dd5d', pr_url=None, pr_revision=None, pr_num=None)